In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms 


In [3]:
X = torch.randn(1000000,10 , dtype=torch.float32 , requires_grad=True)
Y = torch.randn(1000000,10 , dtype=torch.float32 , requires_grad=True)
Z = 5* X + 7 *Y + 500

print (Z)

tensor([[515.7012, 490.2841, 510.3097,  ..., 490.2304, 500.2619, 506.8944],
        [494.7331, 509.0645, 501.6894,  ..., 499.8628, 502.1999, 481.8784],
        [505.5822, 500.7718, 503.6093,  ..., 493.6699, 505.8808, 507.2207],
        ...,
        [493.5047, 512.9279, 501.6550,  ..., 503.0477, 489.9249, 495.9157],
        [492.7509, 505.6155, 509.0760,  ..., 500.8198, 500.3739, 498.5204],
        [511.4201, 499.0053, 506.7788,  ..., 506.1491, 493.3253, 503.5687]],
       grad_fn=<AddBackward0>)


In [4]:
class LR (nn.Module ):
    def __init__(self):
        super(LR , self).__init__()
        self.weight1 = nn.Parameter(torch.randn(1, dtype=torch.float32) , requires_grad=True) 
        self.weight2 = nn.Parameter(torch.randn(1, dtype=torch.float32) , requires_grad=True)
        self.bias = nn.Parameter(torch.randn(1, dtype=torch.float32) , requires_grad=True)
        
    def forward(self , X , Y) : 
        return  (self.weight1) * X + (self.weight2) * Y + (self.bias)

In [5]:
torch.manual_seed(42)
model1 = LR()
model1.state_dict()
loss_function = nn.MSELoss()
optimizer = torch.optim.SGD(model1.parameters() , lr=0.001)

model1.eval()    
with torch.inference_mode():
    predict = model1(X,Y)
    
    
X_numpy = X.cpu().detach().numpy().tolist()
Y_numpy = Y.cpu().detach().numpy().tolist()
Z_numpy = Z.cpu().detach().numpy()
predict_numpy = predict.cpu().detach().numpy()

Z_numpy.shape , predict_numpy.shape



((1000000, 10), (1000000, 10))

In [6]:
model2 = LR()


X.shape , Y.shape , Z.shape 

(torch.Size([1000000, 10]),
 torch.Size([1000000, 10]),
 torch.Size([1000000, 10]))

In [12]:
loss_fn = nn.L1Loss()
optimizer = torch.optim.SGD(model2.parameters() , lr=1)

In [8]:
def train_model(model , X , Y , Z , loss_function , optimizer , epoches):
    epoches_list = []
    tr_losses = []
    test_losses= []

    for epoch in range(epoches):
        model.train()
        optimizer.zero_grad()
        predict = model(X,Y)
        tr_loss = loss_function(predict , Z)
        tr_loss.backward(retain_graph=True)
        optimizer.step()
        
        model.eval()
        with torch.inference_mode():
            test_predict = model(X,Y)
            test_loss = loss_function(test_predict , Z)
            
        if epoch % 100 == 0 :
            epoches_list.append(epoch)
            tr_losses.append(tr_loss)
            test_losses.append(test_loss)
        print(f'Epoch {epoch} , Train Loss {tr_loss.item()} , Test Loss {test_loss.item()}')


In [13]:
train_model(model2 , X , Y , Z , loss_fn , optimizer , 100000)

Epoch 0 , Train Loss 471.7832336425781 , Test Loss 470.7831726074219
Epoch 1 , Train Loss 470.7831726074219 , Test Loss 469.783203125
Epoch 2 , Train Loss 469.783203125 , Test Loss 468.783203125
Epoch 3 , Train Loss 468.783203125 , Test Loss 467.7832336425781
Epoch 4 , Train Loss 467.7832336425781 , Test Loss 466.7831726074219
Epoch 5 , Train Loss 466.7831726074219 , Test Loss 465.78314208984375
Epoch 6 , Train Loss 465.78314208984375 , Test Loss 464.78314208984375
Epoch 7 , Train Loss 464.78314208984375 , Test Loss 463.7832336425781
Epoch 8 , Train Loss 463.7832336425781 , Test Loss 462.7831726074219
Epoch 9 , Train Loss 462.7831726074219 , Test Loss 461.783203125
Epoch 10 , Train Loss 461.783203125 , Test Loss 460.78314208984375
Epoch 11 , Train Loss 460.78314208984375 , Test Loss 459.7832336425781
Epoch 12 , Train Loss 459.7832336425781 , Test Loss 458.7832336425781
Epoch 13 , Train Loss 458.7832336425781 , Test Loss 457.783203125
Epoch 14 , Train Loss 457.783203125 , Test Loss 456.

KeyboardInterrupt: 

In [34]:
w1 = round(model2.weight1.cpu().detach().tolist()[0])
w2 = round(model2.weight2.cpu().detach().tolist()[0])
b = round(model2.bias.cpu().detach().tolist()[0])

weights = {}
weights['w1'] = w1
weights['w2'] = w2
weights['b'] = b

weights


{'w1': 5, 'w2': 7, 'b': 499}

In [35]:
torch.save(model2.state_dict() , 'model.pth')